In [1]:
import bpy
from math import *
from mathutils import *
import numpy as np
import pandas as pd

C:\Users\ackee\AppData\Local\Temp\tmp0k6tfpoh\kernel.py:44: DeprecationWarning: There is no current event loop
  loop = asyncio.get_event_loop()


In [2]:
# Variable for currently active object
myobj = bpy.context.object
# Alternatively, if you know that the object is called 'Cube'
# you can reach it by
# myobj = bpy.data.objects['Cube']

# Clear all previous animation data
myobj.animation_data_clear()

# set first and last frame index
total_time = 2*pi # Animation should be 2*pi seconds long
fps = 24 # Frames per second (fps)
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = round(total_time*fps)+1

# loop of frames and insert keyframes at every frame
nlast = bpy.context.scene.frame_end
for n in range(nlast):
    t = total_time*n/nlast

    # Set frame like this
    bpy.context.scene.frame_set(n)

    # Set current location like this
    myobj.location.x = 5*cos(t)
    myobj.location.y = 5*sin(t)
    myobj.location.z = sin(5*t)

    # Insert new keyframe for "location" like this
    myobj.keyframe_insert(data_path="location")

## Task 3: Rigid body dynamics in Blender by Python scripting 

In [76]:
myobj = bpy.context.object
myobj.animation_data_clear()

t_interval = [0,25]
h = 0.01    # step size

q0 = Quaternion((1.0, 0, 0, 0))
w0 = Vector((10e-2, 1, 0))
I = Matrix(((0.5,0,0),(0,2,0),(0,0,4)))

# Lie-Euler time-stepping scheme
# w = w0
# w_next = w - h*I.inverted()(w.cross(I*w))
# R_next = R*Quaternion(w_next)


time_steps = np.arange(t_interval[0], 2.4*t_interval[1]+h, h)   # TODO Look at switching to range?
data = pd.DataFrame(columns=['w', 'q'], index=time_steps)
data.w.iloc[0] = w0
data.q.iloc[0] = q0

# First simulate the data
for i, t in enumerate(time_steps[:-1]):
    w = data.w.iloc[i]
    q = data.q.iloc[i]
    
    data.w.iloc[i+1] = w - h*I.inverted()@(w.cross(I@w))
    # w_next = w - h*I.inverted()*(w.cross(I*w))
    data.q.iloc[i+1] = q@Quaternion(h*data.w.iloc[i+1])
    # q_next = q*Quaternion(w_next)

# print(data.head())
# print(data.tail())

                                                      w  \
0.00                    [0.10000000149011612, 1.0, 0.0]   
0.01  [0.10000000149011612, 1.0, -0.000375000003259629]   
0.02  [0.10001499950885773, 0.9999993443489075, -0.0...   
0.03  [0.10004500299692154, 0.9999980330467224, -0.0...   
0.04  [0.10009000450372696, 0.9999960660934448, -0.0...   

                                                      q  
0.00                               [1.0, 0.0, 0.0, 0.0]  
0.01  [0.9999873638153076, 0.0004999951343052089, 0....  
0.02  [0.9999495148658752, 0.0010000427719205618, 0....  
0.03  [0.9998863935470581, 0.0015001953579485416, 0....  
0.04  [0.9997979998588562, 0.0020005053374916315, 0....  
                                                       w  \
59.96  [0.07282499223947525, -1.1246671676635742, -0....   
59.97  [0.07245615869760513, -1.1246776580810547, -0....   
59.98  [0.07210113853216171, -1.124687671661377, -0.0...   
59.99  [0.07175986468791962, -1.124697208404541, -0.0... 

C:\Users\ackee\AppData\Local\Temp\tmp0k6tfpoh\kernel.py:44: DeprecationWarning: There is no current event loop
  loop = asyncio.get_event_loop()


In [77]:
# Animation 
steps_per_frame = 10
total_time = 25
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = round(total_time*fps)+1
nlast = bpy.context.scene.frame_end
print(len(data))

myobj.rotation_mode = 'QUATERNION'
for n in range(nlast): #range(0,len(data), steps_per_frame): 
    bpy.context.scene.frame_set(n)

    myobj.rotation_quaternion = data.q.iloc[n*10]
    myobj.keyframe_insert(data_path='rotation_quaternion')

6001


In [17]:
def rotation_from_quat(q: Quaternion):
    a = q[0]
    b1 = q[1]
    b2 = q[2]
    b3 = q[3]

    return 2*Matrix((
        ((a**2+b1**2-b2**2-b3**2)/2, b1*b2-a*b3, b1*b3+a*b2),
        (b1*b2+a*b3, (a**2-b1**2+b2**2-b3**2)/2, b2*b3-a*b1),
        (b1*b3-a*b2, b2*b3+a*b1, (a**2-b1**2-b2**2+b3**2)/2)
    ))

print(rotation_from_quat(q0))

<Matrix 3x3 (1.0000, 0.0000, 0.0000)
            (0.0000, 1.0000, 0.0000)
            (0.0000, 0.0000, 1.0000)>
